# Multi-Class Classification & Multi Label Classification Demonstrations

As a clothing retailer with an e-Commerce presence, it's important for us to understand what customers are saying about our products, which products are most popular with our customers, and how we can improve our offerings.



In [25]:
# Gereral Imports
import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from timeit import timeit
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.compose import ColumnTransformer
from sklearn.multiclass import OneVsRestClassifier

# XGBoost
import xgboost as xgb

# Model Persistence Imports
from joblib import dump, load

# Test Processing Imports
import nltk
from nltk import word_tokenize
from nltk.stem import PorterStemmer

# SMOTE Imports
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Plotting Imports
import matplotlib.pyplot as plt
%matplotlib inline

# Preprocessing & EDA

Importing our dataset and providing the necessary cleaning and analysis.

In [26]:
df = pd.read_csv("ClothingReviews.csv")
df.head()

,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Department Name,Class Name
0,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Intimate,Intimates
1,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,Dresses,Dresses
2,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,Dresses,Dresses
3,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,Bottoms,Pants
4,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,Tops,Blouses


## Data Cleaning

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Age                      23486 non-null  int64 
 1   Title                    19676 non-null  object
 2   Review Text              22641 non-null  object
 3   Rating                   23486 non-null  int64 
 4   Recommended IND          23486 non-null  int64 
 5   Positive Feedback Count  23486 non-null  int64 
 6   Department Name          23472 non-null  object
 7   Class Name               23472 non-null  object
dtypes: int64(4), object(4)
memory usage: 1.4+ MB


In [28]:
df.shape

(23486, 8)

### Null Values

Nulll values are generally not desireable in a dataset.  In certain cases, observations (rows) with low counts will simply be dropped, in other cases, they can be filled with other values.

In [29]:
# fill the NA values with empty string so that we don't drop rows that are only missing a title
df['Title'].fillna('', inplace=True)

In [30]:
# count of nulls
df.isnull().sum()

Age                          0
Title                        0
Review Text                845
Rating                       0
Recommended IND              0
Positive Feedback Count      0
Department Name             14
Class Name                  14
dtype: int64

In [31]:
df.dropna(inplace=True)

In [32]:
df['Text'] = df['Title'] + ' ' + df['Review Text']

In [33]:
# Add column 'text_len' that counts the length for the derived field
df['text_len'] = df.apply(lambda row: len(row['Text']), axis = 1)

In [34]:
df.drop(columns=['Title', 'Review Text'], inplace=True)

### Duplicates

A common practice is to review any duplicates.  If there are large quantities, they can skew the results.

In [35]:
len_before = df.shape[0]
df.drop_duplicates(inplace=True)
len_after = df.shape[0]

print("Before =", len_before)
# drop duplicates
print("After =", len_after)
print('')
print("Total Removed =", len_before - len_after)

Before = 22628
After = 22626

Total Removed = 2


## Check for Imbalanced Data

In [36]:
df['Department Name'].value_counts()

Tops        10048
Dresses      6145
Bottoms      3661
Intimate     1652
Jackets      1002
Trend         118
Name: Department Name, dtype: int64

In [37]:
filter = (df['Class Name'] == 'Chemises') | (df['Class Name'] == 'Casual bottoms')
df = df[~filter]

## Text Cleaning

For **Parts** of our analysis, the text needs to have some basic transformation for our models to work propertly.  These are as follows:

1. **Lower**: Convert all characters to lowercase
1. **Remove Punctuation**: In most cases, punctuation doesn't help NLP and ML models and can be removed.
1. **Stop Word Removal**: Stop words generally don't add context to analysis (unless the length of text is very short (`100` - `200` characters) and can be removed.
1. **Lemmatization**: Words will be reduced to there *Lemma* or root.  This will greatly improve the accuracy of the analysis since words like `simming` and `swimmer` will be reduced to `swim`.

**Note**: The orginal text will be preserved for other analysis.

In [38]:
df['Text'][1]

' Love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.'

In [39]:
def process_string(text):
    
    final_string = ""
    
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctionation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    # Remove stop words and usless words
    text = text.split()
    useless_words = nltk.corpus.stopwords.words("english")
    text_filtered = [word for word in text if not word in useless_words]

    # Remove numbers
    text_filtered = [re.sub('\w*\d\w*', '', w) for w in text_filtered]
    
    # Stem the text with NLTK PorterStemmer
    stemmer = PorterStemmer() 
    text_stemmed = [stemmer.stem(y) for y in text_filtered]

    # Join the words back into a string
    final_string = ' '.join(text_stemmed)
    
    return final_string

In [40]:
df['Text_Processed'] = df['Text'].apply(lambda x: process_string(x))

In [41]:
df['Text_Processed'][1]

'love dress sooo pretti happen find store im glad bc never would order onlin bc petit bought petit  love length hit littl knee would definit true midi someon truli petit'

# Sentiment Based Prediction Model:

Next, we'll create a Supervised ML model to predict whether a customer will recommend a product based on the text from the review and the sentiment of that text, and the length of the review.

To create our model, we will be mixing both text and numeric values.  There are multiple ways to accomplish this, but we will be using a `ColumnTransformer` in a Pipeline.

## Pipeline Building

In [42]:
def create_pipe(clf):
    '''Create a pipeline for a given classifier.  The classifier needs to be an instance
    of the classifier with all parmeters needed specified.'''
    
    # Each pipeline uses the same column transformer.  
    column_trans = ColumnTransformer(
            [('Text', TfidfVectorizer(), 'Text_Processed'),
             ('Text Length', MinMaxScaler(), ['text_len'])],
            remainder='drop') 
    
    pipeline = Pipeline([('prep',column_trans),
                         ('clf', clf)])
     
    return pipeline

## Model Selection via Cross-Validation

In [63]:
X = df[['Text_Processed', 'text_len']]
y = df['Class Name']

In [64]:
print(X.shape)
print(y.shape)

(22624, 2)
(22624,)


In [ ]:
models = {'LogReg' : LogisticRegression(random_state=42, 
                                        class_weight='balanced', 
                                        max_iter=500),
          'RandomForest' : RandomForestClassifier(
                                        class_weight='balanced', 
                                        random_state=42),
          'XGBoost' : xgb.XGBClassifier(random_state=42, 
                                        verbosity=0)
          }

for name, model, in models.items():
    clf = model
    pipeline = create_pipe(clf)
    cv = RepeatedStratifiedKFold(n_splits=10, 
                                 n_repeats=3, 
                                 random_state=1)
    %time scores = cross_val_score(pipeline, X, y, scoring='f1_weighted', cv=cv, n_jobs=1, error_score='raise')
    print(name, ': Mean f1 Weighted: %.3f and Standard Deviation: (%.3f)' % \
        (np.mean(scores), np.std(scores)))

## Model Training

In [65]:
# Make training and test sets 
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=53)

## Model Validation

In [66]:
def fit_and_print(pipeline, name):
    ''' take a supplied pipeline and run it against the train-test spit 
    and product scoring results.'''
    
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print(metrics.classification_report(y_test, y_pred, digits=3)) 

In [67]:
clf = LogisticRegression(random_state=42, 
                         class_weight='balanced', 
                         max_iter=500)
pipeline = create_pipe(clf)
fit_and_print(pipeline, 'LogReg')

              precision    recall  f1-score   support

     Blouses      0.545     0.606     0.574       982
     Dresses      0.930     0.798     0.859      2037
  Fine gauge      0.356     0.399     0.376       351
   Intimates      0.278     0.667     0.392        45
     Jackets      0.463     0.698     0.556       205
       Jeans      0.782     0.771     0.777       354
       Knits      0.685     0.464     0.554      1591
    Layering      0.073     0.344     0.121        32
     Legwear      0.430     0.702     0.533        57
      Lounge      0.182     0.260     0.215       215
   Outerwear      0.532     0.536     0.534       110
       Pants      0.657     0.698     0.676       463
      Shorts      0.335     0.663     0.445        89
      Skirts      0.756     0.875     0.811       287
       Sleep      0.222     0.301     0.256        73
    Sweaters      0.555     0.483     0.517       437
        Swim      0.733     0.810     0.769       105
       Trend      0.015    

## Persist the Model

In [52]:
dump(pipeline, 'multiclass.joblib') 

['multiclass.joblib']

## Test on Custom Data

In [53]:
# Load the model
pipeline = load('multiclass.joblib')

In [54]:
def create_test_data(x):
    
    x = process_string(x)
    length = len(x)
    
    d = {'Text_Processed' : x,
        'text_len' : length}

    df = pd.DataFrame(d, index=[0])
    
    return df

In [55]:
revs = ['This dress is gorgeous and I love it and would gladly reccomend it to all of my friends.',
        'This skirt has really horible quality and I hate it!',
        'A super cute top with the perfect fit.',
        'The most gorgeous pair of jeans I have seen.',
        'this item is too little and tight.']

In [56]:
print('Returns 1 for Positive reviews and 0 for Negative reviews:','\n')
for rev in revs:
    c_res = pipeline.predict(create_test_data(rev))
    print(rev, '=', c_res)

Returns 1 for Positive reviews and 0 for Negative reviews: 

This dress is gorgeous and I love it and would gladly reccomend it to all of my friends. = ['Dresses']
This skirt has really horible quality and I hate it! = ['Skirts']
A super cute top with the perfect fit. = ['Knits']
The most gorgeous pair of jeans I have seen. = ['Jeans']
this item is too little and tight. = ['Legwear']


## Multi-Label Classification

Next, we'll attempt to create a Supervised Machine Learning model to classify products by Department.  This process looks at the text that a user wrote in a review and determined its Department.

An interesting opportunity is to use this information to **cross-sell** or **cross-list** products.  If there is a strong enough probability that an item could be in multiple departments from our analysis, could we **increase sales** with cross-marketing?

In [ ]:
# Tokenize the words
df['Department Name'] = df['Department Name'].apply(word_tokenize)

In [ ]:
X = df[['Text_Processed', 'Department Name']]
y = df['Department Name']

In [ ]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

In [ ]:
print(mlb.classes_)

In [ ]:
def create_pipe(clf):
    
    # Create the column transfomer
    column_trans = ColumnTransformer(
            [('Text', TfidfVectorizer(), 'Text_Processed')],
            remainder='drop') 
    
    # Build the pipeline
    pipeline = Pipeline([('prep',column_trans),
                         ('over', SMOTE(random_state=42)),
                         ('under', RandomUnderSampler(random_state=42)), 
                         ('clf', clf)])
     
    return pipeline

In [ ]:
models = {'Logistic Regression' : OneVsRestClassifier(LogisticRegression(max_iter=500, random_state=42)),
          'Random Forest' : OneVsRestClassifier(RandomForestClassifier(random_state=42)),
          'XGB' : OneVsRestClassifier(xgb.XGBClassifier(random_state=42, verbosity=0))
          }

for name, model, in models.items():
    clf = model
    pipeline = create_pipe(clf)
    %time scores = cross_val_score(pipeline, X, y, scoring='f1_macro', cv=3, n_jobs=1, error_score='raise')
    print(name, ': Mean f1 Macro: %.3f and Standard Deviation: (%.3f)' % (np.mean(scores), np.std(scores)))

## Model Building & Validation

In [ ]:
# Make training and test sets 
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.33, 
                                                    random_state=1)

In [ ]:
print(y_train.shape)
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Note: Optimization was performed prior to Hyperparemeter selection
clf = OneVsRestClassifier(xgb.XGBClassifier(random_state=42, verbosity=0))
pipeline = create_pipe(clf)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
score = metrics.f1_score(y_test, y_pred, average='macro', zero_division=0)

print(metrics.classification_report(y_test, y_pred, digits=3, zero_division=0))

In [ ]:
# Retreive the text lables from the MultiLabelBinarizer
pred_labels = mlb.inverse_transform(y_pred)

# Append them to the DataFrame
X_test['Predicted Labels'] = pred_labels

In [ ]:
# Display a random sample of them
pd.set_option('display.max_colwidth', None)
X_test.sample(10, random_state=0)

**Observations:**

 - In many cases, our classifier was very accurate in determining the correct Department.  Since we used a multi-label classifier, the algorithm suggested more than one label in some cases.  Upon inspecting these, in some cases, the suggestion of an additional Department is incredibly logical.  
 - One item in the above list did not receive a predicted label.
 - Two are showing multiple classes, but in one of these cases, they make sense.  `Jackets` and `Tops`.  For the other, it's classifying it as `Intimate` and `Tops`, which probably would not apply.
 - There is one misclassified item.  The original class was `Trend`, and the predicted class is `Bottoms` for this skirt.  However, a skirt is a bottom and therefore is correct.  We also do not have many observations in the `Trend` class; therefore, it's harder to be accurate with predictions here, as we noted earlier.
 - It's possible to use these multi-label classes to investigate **cross-marketing / cross-listing** opportunities for these products.  

## Persist the Model

Saving both the pipeline as well as the `MultiLabelBinarizer`.

In [ ]:
dump(mlb, 'mlb.joblib') 
dump(pipeline, 'multiclass.joblib') 

## Test on New Data

In [ ]:
def create_test_data(x):
    '''create a dataframe that is in the same format as the Train and Test Data.
    ['Bottoms' 'Dresses' 'Intimate' 'Jackets' 'Tops' 'Trend']'''
    
    s = process_string(x[0])
    
    d = {'Text_Processed' : s,
         'Department Name' : x[1]}

    df = pd.DataFrame(d, index=[0])
    
    return df

In [ ]:
revs = [('This dress is gorgeous and would gladly reccomend it to all of my friends.', ['Dresses']),
     ('These pants have really horible quality and I hate them!', ['Bottoms']),
     ('A super cute blouse with a great fit.', ['Tops']),
     ('The most gorgeous pair of jeans I have seen.', ['Bottoms']),
     ('This bra is silky smooth material and fits perfectly.', ['Intimate'])]

In [ ]:
for rev in revs:
    c_res = pipeline.predict(create_test_data(rev))
    print(rev[0], '\n', rev[1], '\n' ,mlb.inverse_transform(c_res), '\n')